In [1]:
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv
import base64

/opt/homebrew/anaconda3/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Creating a Pinecone Index

In [2]:
load_dotenv() 

True

In [3]:
os.getenv("PINECONE_API_KEY")

'24e23d1d-b502-4e85-a367-8f8b597fddfe'

In [4]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name=os.getenv("INDEX_NAME")
index_list = [index.name for index in pc.list_indexes()]

spec=ServerlessSpec(
    cloud="aws",
    region=os.getenv("PINECONE_ENVIRONMENT")
)

In [5]:
region=os.getenv("PINECONE_ENVIRONMENT")
region

'us-east-1'

In [ ]:
if index_name not in index_list:
    pc.create_index(
        name = index_name,
        dimension = 1024,
        metric = "cosine",
        spec = spec
    )
else: 
    index = pc.Index(index_name)
    dimension = index.describe_index_stats()["dimension"]
    if dimension != 1024:
        pc.delete_index(index_name)
        pc.create_index(
            name = index_name,
            dimension = 1024,
            metric = "cosine",
            spec = spec
        )

# PDF Information Extraction

In [9]:
from mistralai import Mistral
from mistralai.models import OCRResponse
from IPython.display import Markdown, display

In [10]:
api_key = os.environ.get("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

In [15]:

def ocr_response(file_path: str) -> OCRResponse:
    '''
    This function takes a file path and returns the OCR response.
    '''
    uploaded_pdf = client.files.upload(
        file={
            "file_name": os.path.basename(file_path),
            "content": open(file_path, "rb"),
        },
    purpose="ocr"
    )  

    # Getting the signed URL for the uploaded PDF
    signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

    # To extract text from the uploaded PDF using OCR
    ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": signed_url.url,
      },
      include_image_base64=True
    )

    # response_dict = json.loads(ocr_response.model_dump_json())
    return ocr_response

In [16]:
def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    """
    Replace image placeholders in markdown with base64-encoded images.

    Args:
        markdown_str: Markdown text containing image placeholders
        images_dict: Dictionary mapping image IDs to base64 strings

    Returns:
        Markdown text with images replaced by base64 data
    """
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str

def get_combined_markdown(ocr_response: dict) -> str:
    """
    Combine OCR text and images into a single markdown document.

    Args:
        ocr_response: Response from OCR processing containing text and images

    Returns:
        Combined markdown string with embedded images
    """
    markdowns: list[str] = []
    # Extract images from page
    for page in ocr_response.pages:
        image_data = {}
        for img in page.images:
            image_data[img.id] = img.image_base64
        # Replace image placeholders with actual images
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))

    return "\n\n".join(markdowns)

In [23]:
dir_name = 'pdf_collection'
syllabus_path = os.path.join(dir_name, os.listdir(dir_name)[3])
complex_graph_path = os.path.join(dir_name, os.listdir(dir_name)[2])
syllabus_path, complex_graph_path

('pdf_collection/syllabus.pdf', 'pdf_collection/complex_graph.pdf')

In [17]:
syllabus_ocr = ocr_response(syllabus_path)

In [22]:
syllabus_ocr_images = get_combined_markdown(syllabus_ocr)
syllabus_ocr_images

"The information contained on this page is designed to give students a representative example of material covered in the course. Any information related to course assignments, dates, or course materials is illustrative only.\n\n# (2) NYU <br> TANDON SCHOOL OF ENGINEERING \n\n## Course Syllabus\n\nComputer Science and Engineering\nPrinciples of Database Systems\n\n## Course Information\n\n## Course Prerequisites\n\nGraduate student status.\n\n## Course Description\n\nThis course broadly introduces database systems, including the relational data model, query languages, database design, index and file structures, query processing and optimization, concurrency and recovery, transaction management and database design. Students acquire hands-on experience in working with database systems and in building web-accessible database applications.\n\n## Course Objectives\n\nThis course will provide students with the opportunity to:\n\n- Apply queries in relational algebra to retrieve data.\n- Apply

In [19]:
display(Markdown(get_combined_markdown(syllabus_ocr)))

The information contained on this page is designed to give students a representative example of material covered in the course. Any information related to course assignments, dates, or course materials is illustrative only.

# (2) NYU <br> TANDON SCHOOL OF ENGINEERING 

## Course Syllabus

Computer Science and Engineering
Principles of Database Systems

## Course Information

## Course Prerequisites

Graduate student status.

## Course Description

This course broadly introduces database systems, including the relational data model, query languages, database design, index and file structures, query processing and optimization, concurrency and recovery, transaction management and database design. Students acquire hands-on experience in working with database systems and in building web-accessible database applications.

## Course Objectives

This course will provide students with the opportunity to:

- Apply queries in relational algebra to retrieve data.
- Apply queries in SQL to create, read, update and delete data in a database.
- Apply the concepts of entity integrity constraint and referential integrity constraint (including definition of the concept of a foreign key).
- Describe the normal forms (1NF, 2NF, 3NF, BCNF, and 4NF) of a relation.
- Apply normalization to a relation to create a set of BCNF relations and denormalize a relational schema.

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System

- Describe functional dependency between two or more attributes that are a subset of a relation.
- Understand multi-valued dependency and identify examples in relational schemas.
- Sketch conceptual data models (including ER) to describe a database structure.
- Apply SQL to create a relational database schema based on conceptual and relational models.
- Apply stored procedures, functions, and triggers using a commercial relational DBMS.
- Describe concurrency control and how it is affected by isolation levels in the database.
- Analyze Current Research in Database Systems.


## Course Structure

This course is conducted entirely online, which means you do not have to be on campus to complete any portion of it. You will participate in the course using NYU Classes located at https://newclasses.nyu.edu Your final grade will be computed as a combination of the components shown below.

- Quizzes: 30\%
- Labs: $40 \%$
- Project: $30 \%$


## Weekly Structure

Week 1: Introduction to the Relational Model

- Introduce class and overview of course topics.

Weeks 2-4: SQL Language

# NYU TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System

- Introduction to SQL
- Intermediate SQL
- Advanced SQL

Week 5: Formal Relational Query Languages

- Relational Algebra
- Tuple Relational Calculus
- Domain Relational Calculus

Week 6: Database Design: The Entity-Relationship Approach

- ER Design
- Reduction to Relational Model

Week 7: Relational Database Design

- Functional Dependency
- Multivalued Dependency
- Normal Forms.

Week 8: Application Design

- Web Architectures
- Application Security

Week 9: Storage and File Structure

- Physical Storage
- Record Organization

Week 10: Indexing and Hashing

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System

- Ordered Indices
- Hashed Indices
- Bitmap Indices

Weeks 11-12: Query Processing \& Optimization

- Query Processing
- Query Optimization

Week 13: Transactions \& Concurrency Control

- ACID Properties
- Transaction Management

Week 14: Recovery System \& Database System Architectures

- Locks
- Deadlocks
- Snapshot Isolation

Week 15: Student Presentations

- Presentations and reviews


## Learning Time Rubric

Please modify the below table to represent the breakdown of learning time in each week of your course.

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System

| Learning <br> Time <br> Element | Asynchronous* / <br> Synchronous** | Time on <br> Task for <br> Students <br> (weekly) | Notes |
| :--: | :--: | :--: | :-- |
| Reading <br> Assignments <br> / Recorded <br> Lecture | Asynchronous | 2.5 hours | Video format. <br> Expect quizzes <br> throughout the <br> module or <br> weekly chapter <br> readings |
| Weekly <br> Discussion <br> Board \& Peer <br> Review | Asynchronous | 1.5 hours | Students are <br> expected to post <br> responses to <br> weekly topic <br> questions. See <br> Interaction <br> Policy. |
| Assessment <br> (Labs and <br> Programming <br> assignments) | Asynchronous | 2 hours | Students submit <br> their assignment <br> by [the end of <br> the week] |
| Reading <br> Assignment | Asynchronous | 2 hours | Reading <br> assigned <br> textbook <br> chapters and <br> journal articles. |
| Live webinars | Synchronous | 2 hours | Group <br> discussion in <br> class, live, <br> overly weekly <br> chapter |

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System

## Course Communication

## Interaction Policy

Please follow the interaction guidelines stated below for this course.

- I will be holding online virtual classroom sessions every week. This virtual classroom will be held via NYU Classes on Thursdays from 8am to 9am.
- The course will involve regular discussions via the Discussion Forums within NYU Classes and students are encouraged to participate.
- If you have a technical or course content related question, please send me an email. If I think that your question can benefit the class, I might post it on the discussion forum.
- If you have a question related to grading, please send an email to the TA and cc on the email thread. The TA will be responsible for examining your answers and providing a grade as per my guidelines.
- If any other questions need to be answered that are not addressed via email or the live classroom, I can hold virtual office hours on an appointment basis.


## Announcements

Announcements will be posted on NYU Classes on a regular basis. You can locate all class announcements under the Announcements tab of our class. Be sure to check the class announcements regularly as they will contain important information about class assignments and other class matters.

## Email

You are encouraged to post your questions about the course in the Forums discussions on NYU Classes. This is an open forum in which you and your classmates are encouraged to answer each other's questions. But, if you need to contact me directly, please email me. All

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System
homework, labs or programming assignments related questions must be researched first on own time, then posted on forums, then discussed with TAs during weekly reviews, and then can be forwarded to me. Typically, you can expect a response within 48 hours.

## Readings

Avi Silberschatz, Henry F. Korth,S. Sudarshan, Database System Concepts, Sixth Edition, McGraw Hill

You can access NYU's central library here: http://library.nyu.edu/
You can access NYU Tandon's Bern Dibner Library here:
http://library.poly.edu/

RECOMMENDED READINGS are online journal articles provided in each lecture You can access NYU's central library here: http://library.nyu.edu/

You can access NYU Tandon's Bern Dibner Library here:
http://library.poly.edu/

## Assignments and Exams

## Exams Administered and Proctored Online

Exams in this course are administered through NYU Classes. You are required to arrange an online proctor for your exams via ProctorU. More information on ProctorU and scheduling proctoring sessions can be found on Tandon Online's website.

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System

## Exams Administered On Paper and Proctored Remotely

Exams in this course are administered via paper and pencil. If you are not able to attend an exam session on-campus, you are required to secure in-person proctoring arrangements near your location. Tandon Online's website.

## University Policies

## Moses Center Statement of Disability

Academic accommodations are available for students with disabilities. Please contact the Moses Center for Students with Disabilities (212-998-4980 or mosescsd@nyu.edu) for further information. Students who are requesting academic accommodations are advised to reach out to the Moses Center as early as possible in the semester for assistance.

## NYU Tandon School of Engineering Policies and Procedures on Academic Misconduct ${ }^{1}$

A. Introduction: The School of Engineering encourages academic excellence in an environment that promotes honesty, integrity, and fairness, and students at the School of Engineering are expected to exhibit those qualities in their academic work. It is through the process of submitting their own work and receiving honest feedback on that work that students may progress academically. Any act of academic dishonesty is seen as an attack upon the School and will not be tolerated. Furthermore, those who breach the School's rules on academic integrity will be sanctioned under this Policy. Students are responsible for familiarizing themselves with the School's Policy on Academic Misconduct.
B. Definition: Academic dishonesty may include misrepresentation, deception, dishonesty, or any act of falsification committed by a student to influence a grade or other academic evaluation. Academic dishonesty also includes intentionally damaging the academic work of others or assisting other students in acts of dishonesty. Common

[^0]
[^0]:    ${ }^{1}$ Excerpted from the Tandon School of Engineering Student Code of Conduct

# NYU <br> TANDON SCHOOL OF ENGINEERING 

Course Syllabus - CS GY 6083 Principles of Database System
examples of academically dishonest behavior include, but are not limited to, the following:
a. Cheating: intentionally using or attempting to use unauthorized notes, books, electronic media, or electronic communications in an exam; talking with fellow students or looking at another person's work during an exam; submitting work prepared in advance for an in-class examination; having someone take an exam for you or taking an exam for someone else; violating other rules governing the administration of examinations.
b. Fabrication: including but not limited to, falsifying experimental data and/or citations.
c. Plagiarism: intentionally or knowingly representing the words or ideas of another as one's own in any academic exercise; failure to attribute direct quotations, paraphrases, or borrowed facts or information.
d. Unauthorized collaboration: working together on work that was meant to be done individually.
e. Duplicating work: presenting for grading the same work for more than one project or in more than one class, unless express and prior permission has been received from the course instructor(s) or research adviser involved.
f. Forgery: altering any academic document, including, but not limited to, academic records, admissions materials, or medical excuses.

In [ ]:
display